# Read CSV into a data frame
In this step we read the CSV into a data frame and do some basic cleanup steps. 




In [5]:
#spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import lit

sc.setLogLevel("WARN")

#Read a file and then write it to the SQL table
datafile = "/user/hive/warehouse/source/source_new"
#datafile = "/user/hive/warehouse/sourcesue2"
df = spark.read.format('parquet').load(datafile)
#df.insert(len(df.columns), 'batch_id', 0, allow_duplicates=True)

#this should work to add a column to the df 
df = df.withColumn('batch_id', lit(0))
df.show(1)
df.printSchema()

+----------------+-------+------------------+------------------+--------------+------+-----------------+-----------------+------------------+------------------+-------------------+-------------------+-------------------+----------------+----------------------------+-------------------------------+-------------------------+----------------------------+------------------+---------------------+-----------------+-----------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+----------------------+-----------------------+----------------------+-----------------------+------------------+------------------+-------------------+-----------------+--------------------+--------------------+------------------+--------------------+------------------+----------------+---------------------+---------------------+------------------------+-------------------------+------------------------+-------------------------+--------------

# (PART 2) Write and READ to Data Pool external Tables in Big Data Cluster
- Write dataframe to SQL external table in Data Pools in Big Data Cluste
- Read SQL external Table to Spark dataframe

In [6]:
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "LSST"
url = servername + ";" + "databaseName=" + dbname + ";"

dbtable = "SourceDP"
user = "sa"
password = "fooRiuzg54" # Please specify password here

datapool_table = "SourceDP"
datasource_name = "SqlDataPool"

batchsize = 1000000


try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .option("batchsize",batchsize)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")


Use MSSQL connector to write to master SQL instance 
MSSQL Connector write(overwrite) to data pool external table succeeded

In [8]:

#testing on small batch of parquet files
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "LSST"
url = servername + ";" + "databaseName=" + dbname + ";"

dbtable= "Source_test"
#dbtable = "SourceDP"
user = "sa"
password = "fooRiuzg54" # Please specify password here

datapool_table = "Source_test"
datasource_name = "SqlDataPool"

batchsize = 1000000


try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .option("batchsize",batchsize)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")